# <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:200%; text-align:center;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0">Santa 2023 - Bidirectional Brute Force with Move Priority</div>
#### <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:150%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >TABLE OF CONTENTS<br><div>
* [IMPORTS](#1)
* [LOAD DATA](#2)
* [FUNCTIONS](#3)
* [SOLVE](#4)

Code modified from: https://www.kaggle.com/code/dinhttrandrise/bidirectional-brute-force-w-wildcards

<a id="1"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" > IMPORTS<br><div> 

In [1]:
import os
import time


import pandas as pd
from ast import literal_eval
from dataclasses import dataclass
import random
from sympy.combinatorics import Permutation
from typing import Dict, List
import zipfile
import numpy as np
import sqlite3

In [2]:
database_file = '../solutions.db'
solution_method = 'bidirectional brute force w/ move priority'
# Connect to the SQLite database
conn = sqlite3.connect(database_file)
cursor = conn.cursor()

select_query = "SELECT id FROM solutions WHERE solution_method <> ? ORDER BY count"
    
# Execute the query
cursor.execute(select_query, (solution_method,))
unsolved_ids = cursor.fetchall()


# Commit the changes and close the connection
conn.commit()
conn.close()



# RUN_TIME = 60 * 60 * 24 * 7
# START_TIME = time.time()
# TIMEOUT = RUN_TIME
# INCLUDES = [id[0] for id in unsolved_ids]

RUN_TIME = 60 * 60 * 11.5
START_TIME = time.time()
TIMEOUT = 60 * 60 * 11.5
TRACK_TIME = 30
MAX_CNT_RATIO = 200
MAX_CNT_RATIO = None
DEBUG_TRACK = True
DEBUG_LOOP = True

INCLUDES = [368]
EXCLUDES = None
MIN_SIZE = None
MAX_SIZE = None



<a id="2"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >LOAD DATA<br><div> 

In [3]:
with zipfile.ZipFile('../../../res/data/santa-2023.zip', 'r') as z:
    
    with z.open('puzzle_info.csv') as f:
        puzzle_info = pd.read_csv(f, index_col = 'puzzle_type')        
                
    with z.open('puzzles.csv') as f:
        puzzles = pd.read_csv(f, index_col = 'id')
    
    with z.open('sample_submission.csv') as f:
        submission = pd.read_csv(f)

<a id="3"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >FUNCTIONS<br><div> 

In [4]:
def init_reverse_moves(moves):
    new_moves = {}
    
    for m in moves.keys():
        new_moves[m] = moves[m]
        xform = moves[m]
        m_inv = '-' + m
        xform_inv = len(xform) * [0]
        for i in range(len(xform)):
            xform_inv[xform[i]] = i
        new_moves[m_inv] = xform_inv

    return new_moves

def apply_move(move, state):

    m = move
    s = state.split(';')

    move_list = moves[m]
    new_state = []
    for i in move_list:
        new_state.append(s[i])
    s = new_state

    return ';'.join(s)

def reverse_move(move, state):
    m = move[1:] if move[0] == '-' else '-' + move
    return apply_move(m, state)

def expand(paths, reverse=False):
    global moves, source_paths, dest_paths, initial_state, solution_state, initial_score, solution_score, prv_size    
    start_tm = time.time() - TRACK_TIME
    states = list(paths.keys())
    initial_min = initial_score
    solution_min = solution_score
    
    fnd_cnt = 0
    loop_cnt = 0
    loop_max = len(initial_state.split(';')) * 10
    loop_gap = 0
    while fnd_cnt == 0 and ((reverse == False and initial_min >= initial_score) or (reverse == True and solution_min >= solution_score)) and loop_cnt < loop_max:
        cnt = 0
        for s in states:
            cnt += 1
            if time.time() - start_tm > TRACK_TIME:
                if DEBUG_TRACK:
                    print('=> [A] ' + str(cnt) + ' / ' + str(len(states)) + ' --> ' + str(initial_score) + ', ' + str(solution_score) + ' -> ' + str(initial_min) + ', ' + str(solution_min) + ' ---> ' + str(len(source_paths.keys())) + ' / ' + str(len(dest_paths.keys())))
                start_tm = time.time()

            if len(paths[s]) + 1 > prv_size:
                continue
                
            for m in moves:
                if reverse:
                    next_s = reverse_move(m, s)
                    score = get_solution_score(next_s)
                    if score < solution_score + loop_gap:
                        if not next_s in paths:
                            paths[next_s] = paths[s] + [m]
                            if score < solution_min:
                                solution_min = score
                            fnd_cnt += 1
                else:
                    next_s = apply_move(m, s)
                    score = get_initial_score(next_s)
                    if score < initial_score + loop_gap:
                        if not next_s in paths:
                            paths[next_s] = paths[s] + [m]
                            if score < initial_min:
                                initial_min = score
                            fnd_cnt += 1

            overlap = list(set(source_paths.keys()).intersection(set(dest_paths.keys())))
            if len(overlap) > 0:
                initial_score = initial_min
                solution_score = solution_min
                return

        loop_gap += 1
        if loop_gap > loop_max:
            loop_gap = loop_max
        loop_cnt += 1

    initial_score = initial_min
    solution_score = solution_min
            
def get_initial_score(state):
    global solution_state    
    return sum(not(s == t) for s, t in zip(solution_state.split(';'), state.split(';')))

def get_solution_score(state):
    global initial_state    
    return sum(not(s == t) for s, t in zip(initial_state.split(';'), state.split(';')))

In [5]:
def solve(pid, prev_solution):
    global moves, source_paths, dest_paths, initial_state, solution_state, initial_score, solution_score, prv_size
    
    ddf = puzzles.loc[pid]
    puzzle_type = ddf['puzzle_type']
    solution_state = ddf['solution_state']
    initial_state = ddf['initial_state']
    num_wildcards = ddf['num_wildcards']

    idf = puzzle_info.loc[puzzle_type]
    allowed_moves = idf['allowed_moves'] 
    moves = literal_eval(allowed_moves)

    moves = init_reverse_moves(moves)

    initial_score = len(solution_state.split(';'))
    solution_score = len(solution_state.split(';'))
        
    prv_size = len(prv_solution.split('.'))
    
    max_cnt = None
    if MAX_CNT_RATIO is not None:
        max_cnt = solution_score * MAX_CNT_RATIO
    
    source_paths = {
        initial_state: []
    }
    dest_paths = {
        solution_state: []
    }

    start_time = time.time()
    solution = None
    count = 0
    while time.time() - START_TIME < RUN_TIME and time.time() - start_time < TIMEOUT:
        count += 1

        if max_cnt is not None and count > max_cnt:
            break
            
        if count % 2:
            expand(source_paths)
        else:
            expand(dest_paths, reverse=True)

        overlap = list(set(source_paths.keys()).intersection(set(dest_paths.keys())))
        if DEBUG_LOOP:
            print('=> [B] ' + str(count) + ' --> ' + str(initial_score) + ' : ' + str(solution_score) + ' --> ' + str(len(source_paths.keys())) + ' / ' + str(len(dest_paths.keys())))
        if len(overlap) > 0:
            mn_score = 10000000
            mn_sol = None
            for oi in range(len(overlap)):
                oe = overlap[oi]
                sol = '.'.join(source_paths[oe] + list(reversed(dest_paths[oe])))
                sz = len(sol.split('.'))
                if sz < mn_score:
                    mn_score = sz
                    mn_sol = sol
            solution = mn_sol
            break
            
    if solution is not None:
        bsz = len(prv_solution.split('.'))
        asz = len(solution.split('.'))
        if asz == bsz:
            print('[' + str(pid) + '] Same : ' + str(bsz) + ' -> ' + str(asz))
        elif asz < bsz:
            print('[' + str(pid) + '] Decrease : ' + str(bsz) + ' -> ' + str(asz))            
        else:
            print('[' + str(pid) + '] Increase : ' + str(bsz) + ' -> ' + str(asz))            
            
    return solution

<a id="4"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >SOLVE<br><div> 

In [6]:
moves = None
source_paths = None
dest_paths = None
initial_state = None
solution_state = None
initial_score = None
solution_score = None
prv_size = None

grows = []
all_ids = submission['id']

database_file = '../solutions.db'
# Connect to the SQLite database
conn = sqlite3.connect(database_file)
cursor = conn.cursor()

for pid in all_ids:
    if INCLUDES is not None and pid not in INCLUDES:
        print(f'=> [{pid}] Not included!')
        continue
    if EXCLUDES is not None and pid in EXCLUDES:
        print(f'=> [{pid}] Excluded!')
        continue
        
    # df = data_df[data_df['id'] == pid]
    df = puzzles.loc[pid]
    solution_state = df['solution_state']
    size = len(solution_state.split(';'))
        
    if MIN_SIZE is not None and size <= MIN_SIZE:
        print('=> [' + str(pid) + '] Skipped by lower size!')
        continue

    if MAX_SIZE is not None and size >= MAX_SIZE:
        print('=> [' + str(pid) + '] Skipped by upper size!')
        continue
        
    game_id = int(pid)
    select_query = "SELECT moves, count FROM solutions WHERE id = ?"
        
    # Execute the query
    cursor.execute(select_query, (game_id,))
    response = cursor.fetchone()
    
    best_moves = response[0]
    best_moves_count = response[1]

    prv_solution = best_moves
    
    solution = solve(pid, prv_solution)
    if solution is None:
        print('=> [' + str(pid) + '] Failed!')        
    else:
        print('=> [' + str(pid) + '] Success!')        
        
        grows.append({'id': pid, 'moves': solution})
        
        moves = solution
        move_count = len(moves.split('.'))
        
        if best_moves_count > move_count:
            print(f'Improvement to {game_id}')
            # Insert the moves into the database
            # insert_query = "INSERT OR REPLACE INTO solutions (id, moves, count, solution_method) VALUES (?, ?, ?, ?)"
            # cursor.execute(insert_query, (game_id, moves, move_count, solution_method))
            # conn.commit()
        
# Commit the changes and close the connection
conn.commit()
conn.close()

=> [0] Not included!
=> [1] Not included!
=> [2] Not included!
=> [3] Not included!
=> [4] Not included!
=> [5] Not included!
=> [6] Not included!
=> [7] Not included!
=> [8] Not included!
=> [9] Not included!
=> [10] Not included!
=> [11] Not included!
=> [12] Not included!
=> [13] Not included!
=> [14] Not included!
=> [15] Not included!
=> [16] Not included!
=> [17] Not included!
=> [18] Not included!
=> [19] Not included!
=> [20] Not included!
=> [21] Not included!
=> [22] Not included!
=> [23] Not included!
=> [24] Not included!
=> [25] Not included!
=> [26] Not included!
=> [27] Not included!
=> [28] Not included!
=> [29] Not included!
=> [30] Not included!
=> [31] Not included!
=> [32] Not included!
=> [33] Not included!
=> [34] Not included!
=> [35] Not included!
=> [36] Not included!
=> [37] Not included!
=> [38] Not included!
=> [39] Not included!
=> [40] Not included!
=> [41] Not included!
=> [42] Not included!
=> [43] Not included!
=> [44] Not included!
=> [45] Not included